# Text generation

The goal of this project is to demonstrate text generation using LSTM neural networks.
Our database contains numerous movie plots taken from Wikipedia, so we will generate something similiar.

In [29]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras.models import Sequential
import random 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


In [38]:
data = pd.read_csv("movie_plots.csv")

movie_plots = data['Plot']
text = movie_plots.str.lower()
np.random.seed(1)
print("Max movie plot len: ", movie_plots.map(len).max())
print("Min movie plot len: ", movie_plots.map(len).min())

all_plots = list(movie_plots.values)
sample = random.sample(all_plots, 50)


Max movie plot len:  36773
Min movie plot len:  15


## Tokenize words

Generally in Natural Language Processing projects, the first step is removal of stop words, such as "the", "a", "an", and punctuation. We will skip this step since we want to generate human-like speech.
Tokenization is turning unique words into unique integers. This step is necessary for preparing data for embedding layer.

In [39]:
max_words = 50000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(sample)

sequences = tokenizer.texts_to_sequences(sample)
sequences = pad_sequences(sequences, maxlen = 80, truncating = 'post')
sequences.shape
print(sequences[:2])

[[ 673    3  432 1855 1856 1857    3 1858 1859   31    4  142    3  129
  1165 1860  433   25    1 1861    5   27  851   29 1862    4   42  238
   432  163  852 1863 1864 1166 1865 1167 1166 1866   19  853   10  432
   674   74   38 1168  675  432   90    2  208    1 1867    4  524   23
   673   20   30   11  164  108  432 1169    4  434   16 1868   42  238
    23   18    1  854 1869   38   14   27  165  268]
 [ 189   31  437 1173    3   35  122   53   31  317  165  130   20    1
   122  239    2 1174  270   35    5   34   17    4  438  268   19   83
   109    9   39  680   96    9 1886  271   46    4  240   12    9  142
     3  212   18    1  122  527    2    9  142    3  528   16   80   11
  1887    9    2    4  272   19   83   80    9  360 1888    1  272    7
   110   35   20  143   68    6 1175   11  529   13]]


In [40]:
# making a single list of tokens so we can apply sliding windows

text = [item for sublist in sequences for item in sublist]
print(len(text))
vocab_size = len(tokenizer.word_index)

4000


In [45]:
print("Vocabulary size: ", vocab_size+1)

# reverse dictionary so we can decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))


Vocabulary size:  4316


### Splitting the data for input and output values

In [42]:
seq_len = 20

dataX = []
dataY = []

for i in range(len(text)-seq_len):
    seq_in = text[i:i+seq_len]
    seq_out = text[i+seq_len]
    dataX.append(seq_in)
    dataY.append(seq_out)
    
size = len(dataX)
print(size)

3980


In [43]:
dataX = np.asarray(dataX)
dataY = np.asarray(dataY)
dataX.shape, dataY.shape

((3980, 20), (3980,))

In [44]:
from keras.utils import np_utils

#trainX = np.reshape(dataX, (size, seq_len, 1))
trainX = dataX
trainy = np_utils.to_categorical(dataY)
trainy.shape

(3980, 4316)

### Creating a model

In [47]:
import keras
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size+1, 10, input_length = trainX.shape[1]))
model.add(LSTM(256))
model.add(Dropout(0.1))
model.add(Dense(trainy.shape[1], activation = 'softmax'))
model.summary()

IndexError: tuple index out of range

In [11]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

filepath = "./weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks = [checkpoint]

hist = model.fit(trainX, trainy, epochs = 20, batch_size = 128, verbose = 1, callbacks = callbacks)


Epoch 1/20
1580/1580 [==============================] - 3s 2ms/step - loss: 6.5144 - acc: 0.1405

Epoch 00001: loss improved from inf to 6.51435, saving model to ./weights.hdf5
Epoch 2/20
1580/1580 [==============================] - 3s 2ms/step - loss: 5.2748 - acc: 0.1715

Epoch 00002: loss improved from 6.51435 to 5.27483, saving model to ./weights.hdf5
Epoch 3/20
1580/1580 [==============================] - 3s 2ms/step - loss: 4.9907 - acc: 0.1772

Epoch 00003: loss improved from 5.27483 to 4.99067, saving model to ./weights.hdf5
Epoch 4/20
1580/1580 [==============================] - 3s 2ms/step - loss: 4.8434 - acc: 0.1766

Epoch 00004: loss improved from 4.99067 to 4.84339, saving model to ./weights.hdf5
Epoch 5/20
1580/1580 [==============================] - 3s 2ms/step - loss: 4.7526 - acc: 0.1747

Epoch 00005: loss improved from 4.84339 to 4.75256, saving model to ./weights.hdf5
Epoch 6/20
1580/1580 [==============================] - 3s 2ms/step - loss: 4.6842 - acc: 0.1747



In [12]:
# Loading weights from a checkpoint

filename = "weights.hdf5"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')


In [13]:
def generate_words(seq, max_len = 20):
    seed = tokenizer.texts_to_sequences([seq])
    print(seed)
    while len(seed[0]) < max_len:
        seed2 = pad_sequences(seed[-19:], maxlen = 19)
        op = model.predict(np.asarray(seed2).reshape(1,-1))
        seed[0].append(op.argmax() + 1)
        
    return " ".join(map(lambda x: reverse_word_map[x], seed[0]))